In [ ]:

import os
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler


# pvlib imports
import pvlib
from pvlib.location import Location
from pvlib.pvsystem import PVSystem
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS



In [ ]:
os.getcwd(
)

In [ ]:
df_meta = pd.read_csv("../../../Input_Data/Meta_Data_PV_Optimised.csv", index_col = 0)

df_meteo = pd.read_csv("../../../Input_Data/ghi_dni_dhi.csv", index_col=0, parse_dates=True)


In [105]:
df_meteo = pd.read_csv("/Users/nikolaushouben/Library/Mobile Documents/com~apple~CloudDocs/PhD_Icloud/PhD_Papers/Paper_2_Collaborative Forecasting/Data/Meteo/irradiance_data-2014_2017.csv", index_col=0, parse_dates=True)

df_meteo.columns = ['ghi', 'dhi', 'dni']

In [156]:
df_meteo = df_meteo.resample("15T").mean()
df_irr = df_meteo
df_irr.index = df_irr.index.tz_localize(tz = None)

In [108]:
df_meta_original = pd.read_csv("../../../Data/System/PV system information dataset TU WIEN.csv", sep = ";", index_col = 0)

In [109]:
df_meta_data = pd.merge(df_meta_original, df_meta, left_index=True, right_index=True)

In [110]:
df_meta_data["tilt"] = df_meta_data["tilt_finetuned"] 
df_meta_data["azimuth"] = df_meta_data["azimuth_finetuned"] 

df_meta_data = df_meta_data.iloc[:,1:-2]
df_meta_data

,latitude,longitude,tilt,azimuth,estimated capacity
151,51.970022,5.329183,18,178,2082.240135
152,52.108729,5.125370,61,201,2762.426588
154,52.090784,5.097724,14,224,281.794994
175,52.087066,5.143054,21,223,284.683805
188,52.099901,5.137199,11,233,1430.592125
192,51.975027,4.934823,61,170,2830.557971
194,52.035392,5.149262,18,216,492.479257
195,52.066426,5.130489,15,186,488.894519
207,52.100728,5.218298,12,185,1926.896860
223,52.095973,5.141322,5,182,3922.004198


In [ ]:

df_power = pd.read_csv("../../../Input_Data/PV_System_Power.csv", index_col=0, parse_dates=True)

In [111]:
def physical_profile(row, df_irr):
    latitude, longitude, tilt, azimuth, capacity = row

    temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS["sapm"][
        "open_rack_glass_glass"
    ]

    location = Location(latitude=latitude, longitude=longitude)

    pvwatts_system = PVSystem(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        module_parameters={"pdc0": capacity, "gamma_pdc": -0.004},
        inverter_parameters={"pdc0": capacity},
        temperature_model_parameters=temperature_model_parameters,
    )

    mc = ModelChain(
        pvwatts_system, location, aoi_model="physical", spectral_model="no_loss"
    )
    mc.run_model(df_irr)
    results = mc.results.ac

    df_results = pd.Series(results)
    df_results.index = df_results.index.tz_localize(None)
    df_results.index.name = "timestamp"
    df_results.name = str(tilt) + ";" + str(azimuth)

    return df_results



In [161]:
import sys


system_id = 487



df_meta_data_id = df_meta_data.loc[system_id]
df_physical_profile = physical_profile(df_meta_data_id, df_irr).to_frame("physical_profile").resample("15T").mean().fillna(method = "bfill", limit = 4)

In [ ]:
df_physical_profile

In [113]:
df_power_id = df_power[[str(system_id)]].resample("15T").mean()

In [138]:
df_compare = pd.merge(df_power_id, df_physical_profile, left_index=True, right_index=True).fillna(method = "bfill", limit= 4).dropna()

In [139]:
from sklearn.preprocessing import MinMaxScaler

In [140]:
scaler = MinMaxScaler()


df_compare[df_compare.columns] = scaler.fit_transform(df_compare[df_compare.columns])

In [142]:
df_compare["diff"] =  df_compare["physical_profile"]- df_compare["487"]
df_compare["norm"] = df_compare["487"]/df_compare["physical_profile"]

In [135]:
px.line(df_compare[:"2015-02-01"])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=487<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '487',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '487',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 15),
                          datetime.datetime(2015, 1, 1, 0, 30), ...,
                          datetime.datetime(2015, 2, 1, 23, 15),
                          datetime.datetime(2015, 2, 1, 23, 30),
                          datetime.datetime(2015, 2, 1, 23, 45)], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=physical_profile<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'physical_profile',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'physical_profile',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 15),
                          datetime.datetime(2015, 1, 1, 0, 30), ...,
                          datetime.datetime(2015, 2, 1, 23, 15),
                          datetime.datetime(2015, 2, 1, 23, 30),
                          datetime.datetime(2015, 2, 1, 23, 45)], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=diff<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'diff',
              'line': {'color': '#00cc96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'diff',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 15),
                          datetime.datetime(2015, 1, 1, 0, 30), ...,
                          datetime.datetime(2015, 2, 1, 23, 15),
                          datetime.datetime(2015, 2, 1, 23, 30),
                          datetime.datetime(2015, 2, 1, 23, 45)], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., ..., 0., 0., 0.]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=norm<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'norm',
              'line': {'color': '#ab63fa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'norm',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 1, 0, 15),
                          datetime.datetime(2015, 1, 1, 0, 30), ...,
                          datetime.datetime(2015, 2, 1, 23, 15),
                          datetime.datetime(2015, 2, 1, 23, 30),
                          datetime.datetime(2015, 2, 1, 23, 45)], dtype=object),
              'xaxis': 'x',
              'y': array([nan, nan, nan, ..., nan, nan, nan]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'index'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

In [145]:
from sklearn.metrics import mean_squared_error, r2_score




In [148]:
mean_squared_error(df_compare["487"], df_compare["physical_profile"], squared=False)


0.06447720657925506

In [149]:
r2_score(df_compare["487"], df_compare["physical_profile"])

0.9242735113595802

In [151]:
from xgboost import XGBRegressor

In [157]:
df_ml = pd.merge(df_compare.iloc[:,:1], df_irr, left_index=True, right_index=True)

In [160]:
df_ml = df_ml.dropna()

In [162]:
location = Location(latitude=df_meta_data_id["latitude"], longitude=df_meta_data_id["longitude"])

In [169]:
df_solar_pos = location.get_solarposition(df_ml.index)[["apparent_zenith", "apparent_elevation", "azimuth",]]

In [172]:
df_ml = pd.concat([df_ml, df_solar_pos], axis = 1)

In [173]:
df_ml

,487,ghi,dhi,dni,apparent_zenith,apparent_elevation,azimuth
2015-01-01 00:00:00,0.000000,0.000000,0.000000,0.0,150.778394,-60.778394,8.193985
2015-01-01 00:15:00,0.000000,0.000000,0.000000,0.0,150.311195,-60.311195,15.147639
2015-01-01 00:30:00,0.000000,0.000000,0.000000,0.0,149.578329,-59.578329,21.881321
2015-01-01 00:45:00,0.000000,0.000000,0.000000,0.0,148.599593,-58.599593,28.323616
2015-01-01 01:00:00,0.000000,0.000000,0.000000,0.0,147.398728,-57.398728,34.428962
...,...,...,...,...,...,...,...
2016-12-09 12:45:00,0.024348,31.702500,32.290110,0.0,76.572055,13.427945,197.246761
2016-12-09 13:00:00,0.028003,37.807270,38.444803,0.0,77.318404,12.681596,200.706548
2016-12-09 13:15:00,0.034113,50.271793,51.183243,0.0,78.192839,11.807161,204.123224
2016-12-09 13:30:00,0.037768,48.945640,49.470127,0.0,79.190165,10.809835,207.492279


In [174]:
df_ml_1 = df_ml.iloc[:,:4]


In [201]:
from sklearn.linear_model import LinearRegression

In [220]:


def get_accuracies_per_month(df, model_type):

    accuracy_dictionary = {}

    start_date = pd.Timestamp("2015-02-01")
    for month in range(1,8):

        train_end = start_date + pd.Timedelta(weeks = 4*month)
        train = df[start_date:train_end]
        test = df[train_end:]


        X_train, y_train = train.iloc[:,1:], train.iloc[:,:1]
        X_test, y_test = test.iloc[:,1:], test.iloc[:,:1]


        model = model_type()
        model.fit(X = X_train, y = y_train)
        y_test["predictions"] = model.predict(X_test)

        rmse = mean_squared_error(y_true=y_test[str(system_id)], y_pred= y_test["predictions"], squared=False)
        r2 = r2_score(y_true=y_test[str(system_id)], y_pred= y_test["predictions"])

        accuracy_dictionary[month] = rmse, r2 

    df_accuracy = pd.DataFrame(accuracy_dictionary, index = ["rmse", "r2"]).T.reset_index()

    return df_accuracy


In [225]:
df_accuracy = get_accuracies_per_month(df_ml_1, LinearRegression)
px.bar(df_accuracy, x = "index", y = ["r2"])

/var/folders/35/267ql_w95qlgmrzwgprns44w0000gn/T/ipykernel_45376/3440725320.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/35/267ql_w95qlgmrzwgprns44w0000gn/T/ipykernel_45376/3440725320.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/35/267ql_w95qlgmrzwgprns44w0000gn/T/ipykernel_45376/3440725320.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'variable=r2<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'r2',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'r2',
              'offsetgroup': 'r2',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([1, 2, 3, 4, 5, 6, 7]),
              'xaxis': 'x',
              'y': array([0.32959972, 0.70157556, 0.84781289, 0.88481471, 0.8992702 , 0.90163827,
                          0.89743965]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'index'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

In [226]:
df_accuracy = get_accuracies_per_month(df_ml, LinearRegression)
px.bar(df_accuracy, x = "index", y = ["r2"])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'variable=r2<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'r2',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'r2',
              'offsetgroup': 'r2',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([1, 2, 3, 4, 5, 6, 7]),
              'xaxis': 'x',
              'y': array([0.33424934, 0.70348054, 0.84817206, 0.88512077, 0.89961748, 0.90198528,
                          0.89780142]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'index'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})